In [1]:
from pydantic import BaseModel
from fastapi import FastAPI, UploadFile, File
from pipeline.chain import Chain
from pipeline.model import Model
from pipeline.db import Db
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from typing import List, Dict, Callable
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
from transformers import StoppingCriteria, StoppingCriteriaList
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import torch
import transformers
import uvicorn
device = 0 if torch.cuda.is_available() else -1  # set to GPU if available
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
print(f"Using device: {device}", torch.cuda.is_available())
device = 'auto'
langchain.debug = True
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import textwrap
import torch
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        #regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        #match = re.search(regex, llm_output, re.DOTALL)

        # if not match:
        #     raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        # action = match.group(1).strip()
        # action_input = match.group(2)
        # Return the action and action input
        # answer = AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

        action, tool_input = llm_output.split("\nAction: ")[1].strip().replace(']', '').split('[')
        answer = AgentAction(tool=action, tool_input=tool_input, log=llm_output)
        return answer



def search_run(input: str) -> str:
    output = search.run(input)
    #answer = qa_chain.search(f'summarize the next text with respect to next request: {input}: {output}')
    return output


def docs_search(input: str) -> str:
    output = qa_chain.search(input)
    #output = Chain.process_llm_response(output, False)
    return output


search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name='Search',
        func=search_run,
        description="useful for when you need to answer questions about current events"
    ),

    Tool(
        name='Lookup',
        func=docs_search,
        description="useful for when you need to answer questions about specific precise information in documents"

    )
]
output_parser = CustomOutputParser()
path = './Training_materials/'


Using device: 0 True
Using device: 0 True


In [2]:
qa_chain = Chain()

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

No sentence-transformers model found with name ./models/e5-large-v2. Creating a new one with MEAN pooling.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [4]:

from torch import cuda, bfloat16
import transformers

model = transformers.AutoModelForCausalLM.from_pretrained(
    'models/mpt-7b-storywriter',
    trust_remote_code=True,
    torch_dtype=bfloat16,
    device_map=device,
    max_length=83968
)

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

In [6]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

# mtp-7b is trained to add "<|endoftext|>" at the end of generations
stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [7]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    device_map=device,
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=1000,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

The model 'MPTForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormFor

In [8]:
pipe_mpt = HuggingFacePipeline(pipeline=generate_text)

In [2]:
# llm_mpt = Model('mpt-7b-storywriter', temperature=0, max_length=2000).llm
# llm_mpt("How can I treat a sprained ankle?")


docs_search('Tell me about Vantage Towers financial situation')


/opt/conda/envs/gpt/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "Tell about Vantage Towers financial situation?",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:StuffDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "question": "Tell about Vantage Towers financial situation?",
  "context": "5/5/2020 GSMA | Case Study: Cellnex - Future Networks\nhttps://www .gsma.com/futurenetworks/wiki/case-study-das-as-a-service/ 15/21CapEx per square meter (up to a factor of x0.8\ncompared to small sites). In any case, Cellnex\nproposes different technological solutions to\nmeet customer expectations at the same time\nthat minimises the cost.\nUser Experience: Moreover, venues will benefit\nfrom a better client/user experience with a unique\nintermediary.\nOperational Management:  At the

' Vantage Towers is a European telecommunications infrastructure company headquartered in Barcelona, Spain. The\ncompany was formed in 2019 through the merger of Cellnex Telecom and Abertis Telecom. As of 2021, it operates\nmore than 70,000 telecom towers across Europe, including Spain, Italy, France, Belgium, Ireland, Switzerland,\nand the Netherlands. The company provides services such as mobile telephony, fixed telephony, fiber optic\nnetworks, and broadcasting. Its revenue in 2020 was approximately €3 billion.'

In [3]:
model = Model()

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
model= Model()
llm = model.llm1()

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [9]:
question_generator = LLMChain(llm=pipe_mpt, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(pipe_mpt, chain_type="map_reduce")

cr_chain = ConversationalRetrievalChain(
    retriever=qa_chain.retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True
)

In [10]:
chat_history = []
query = "What is MNO?"
result = cr_chain({"question": query, "chat_history": chat_history})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "What is MNO?",
  "chat_history": []
}
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:MapReduceDocumentsChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "MNO  Mobile  Network Operator  \nMNO PSU:  MNOs Power Supply Unit(s) for BTS and NTE  per MNO  \nMT Mobile Terminal  \nNDA  Non -Disclosure Agreement  \nNTE:  Network Transmission Equipment for connecting the traffic to the core  \n network, (can be more than one)  per MNO  \n \n2 The NEN - EN 50173 and 50174 also defines locations which can be applicable. For example MER is \ncalled Building Distribution (DB)  and SER is called Floor Distribution (FD).",
      "question": "What is MNO?",
      "chat_history": ""
    },
    {
      "context": "MNO

/opt/conda/envs/gpt/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[llm/end] [1:chain:ConversationalRetrievalChain > 2:chain:MapReduceDocumentsChain > 3:chain:LLMChain > 4:llm:HuggingFacePipeline] [196.89s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\n   * 2.1 General \n    *...\n      * A mobile terminal shall have at least two power supply units connected in parallel with each other or in series depending on the type of equipment used by the operator; these are referred as \"mobile terminals\" hereon after...\n        * 3.3 Network transmission equipments [NTE] may consist of several parts such as base transceiver station [BTS], radio access node [RAN], etc., but they all must comply with this standard....\n     * 4.4 In case of multiple connections between different networks it should not matter where the connection takes place provided that there exists an interface defined within this standard. This means that when using interfaces like GSM-A/GPRS-C, Iu-PSU, E-UTRAN-EPC, UTRAN-GERAN, TETRA-TCCH, WCDMA-UMTS,

Token indices sequence length is longer than the specified maximum sequence length for this model (4680 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2568 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3624 > 1024). Running this sequence through the model will result in indexing errors


[chain/error] [1:chain:ConversationalRetrievalChain > 2:chain:MapReduceDocumentsChain] [200.87s] Chain run errored with error:
"ValueError('A single document was so long it could not be combined with another document, we cannot handle this.')"
[chain/error] [1:chain:ConversationalRetrievalChain] [200.90s] Chain run errored with error:
"ValueError('A single document was so long it could not be combined with another document, we cannot handle this.')"


ValueError: A single document was so long it could not be combined with another document, we cannot handle this.

In [ ]:
result['answer']

In [43]:
doc_chain = load_qa_with_sources_chain(qa_chain.llm, chain_type="map_reduce")
doc_chain

MapReduceDocumentsChain(lc_kwargs={'llm_chain': LLMChain(lc_kwargs={'llm': HuggingFacePipeline(lc_kwargs={'pipeline': <transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f4de54375b0>}, cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f4de54375b0>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None), 'prompt': PromptTemplate(lc_kwargs={'template': 'Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', 'input_variables': ['context', 'question']}, input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\

In [48]:
from langchain.chains.qa_with_sources.map_reduce_prompt import (
    COMBINE_PROMPT,
    EXAMPLE_PROMPT,
    QUESTION_PROMPT,
)

QUESTION_PROMPT

PromptTemplate(lc_kwargs={'template': 'Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', 'input_variables': ['context', 'question']}, input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', template_format='f-string', validate_template=True)

In [45]:
doc_chain.combine_document_chain.lc_kwargs['llm_chain'].prompt.template

'rtgrfdfffdgfsdfd'

In [38]:
doc_chain.combine_document_chain.lc_kwargs['llm_chain'].prompt.lc_kwargs = '''{'template': 'Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: {question}\n=========\n{summaries}\n=========\nFINAL ANSWER:','input_variables': ['summaries', 'question']}'''

In [42]:
doc_chain.combine_document_chain.lc_kwargs['llm_chain'].prompt.lc_kwargs

'{\'template\': \'Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: {question}\n=========\n{summaries}\n=========\nFINAL ANSWER:\',\'input_variables\': [\'summaries\', \'question\']}'

In [2]:

model_name = 'e5-large-v2'
model_path = f'./models/{model_name}'
model_kwargs = {
    'device': 'cuda:0'
}
embeddings = HuggingFaceEmbeddings(model_name=model_path, model_kwargs=model_kwargs)
vectorDB = Chroma(persist_directory='./db', embedding_function=embeddings)

No sentence-transformers model found with name ./models/e5-large-v2. Creating a new one with MEAN pooling.


In [3]:
retriever = vectorDB.as_retriever(search_kwargs={'k': 2})
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7ff5b718dea0>, search_type='similarity', search_kwargs={'k': 2})

In [4]:
retriever.get_relevant_documents('Vantage Towers?')

[]

In [6]:
qa_chain.db.vectorDB

In [7]:
db = Db()

No sentence-transformers model found with name ./models/e5-large-v2. Creating a new one with MEAN pooling.


In [8]:
db.reload_docs_with()

In [8]:
db.retriever.get_relevant_documents('Vantage Towers?')

[]

In [5]:
react = initialize_agent(tools, qa_chain.llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [19]:
qa_chain.db.retriever.get_relevant_documents("What is the most promising direction for investment for the Vantage Towers?")

[Document(lc_kwargs={'page_content': '5/5/2020 GSMA | Case Study: Cellnex - Future Networks\nhttps://www .gsma.com/futurenetworks/wiki/case-study-das-as-a-service/ 7/21stadiums, shopping centres, and skyscrapers. On the road\nto 5G, DAS solutions are continuously evolving to add\nnew functionalities, improve cost-efficiency, reduce\nfootprint and adapt to C-RAN and V-RAN typologies.\nEnd-to-end digital solutions with IT structured cabling is\non the rise. Beyond Passive and Active DAS solutions, a\nnew Advanced DAS  is emerging in the market. This\nsolution is also called DRS, C-RAN DAS, or even assumed\nto be Small Cells by some third parties. Advanced DAS is\nvery low-power solutions that open the door to advanced\nfeatures such as improved positioning by distributing\nradio functionalities to the antenna. There’s also a shift\ntowards C-RAN (Centralised RAN) architecture, where\ncentralised baseband functionality is shared across a\nlarge number of distributed radio nodes to deliver

In [ ]:
#react.run("What is the most promising direction for investment for the Vantage Towers?")

In [ ]:
path = './Training_materials/'
qa_chain = Chain()

In [8]:
def search_run(input: str) -> str:
    output = search.run(input)
    answer = qa_chain.search(f'summarize the next text with respect to next request: {input}: {output}')
    return answer

def docs_search(input: str) -> str:
    output = qa_chain.search(input)
    return output


In [9]:
search = DuckDuckGoSearchRun()


tools = [
    Tool(
        name='search',
        func=search_run,
        description="useful for when you need to answer questions about current events"
    ),

    Tool(
        name='lookup_in_docs',
        func=docs_search,
        description="useful for when you need to answer questions about specific precise information in documents"

    )
]

In [10]:
# template = """Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:
#
# {tools}
#
# Use the following format:
#
# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question
#
# Begin! Remember to answer as a compasionate specialist in cell towers and all related fields when giving your final answer.
#
# Question: {input}
# {agent_scratchpad}"""

In [6]:
# prompt = CustomPromptTemplate(
#     template=template,
#     tools=tools,
#     input_variables=["input", "intermediate_steps"]
# )

In [7]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        #regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        #match = re.search(regex, llm_output, re.DOTALL)



        # if not match:
        #     raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        # action = match.group(1).strip()
        # action_input = match.group(2)
        # Return the action and action input
        # answer = AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

        action, tool_input  = llm_output.split("\nAction: ")[1].strip().replace(']','').split('[')
        answer = AgentAction(tool=action, tool_input=tool_input, log=llm_output)
        return answer

In [8]:
output_parser = CustomOutputParser()

In [15]:
# llm = qa_chain.llm

In [16]:
# llm_chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
# tool_names = [tool.name for tool in tools]
#
# agent = LLMSingleActionAgent(
#     llm_chain=llm_chain,
#     output_parser=output_parser,
#     stop=["\nObservation:"],
#     allowed_tools=tool_names
# )

In [18]:
# agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
#                                                     tools=tools,
#                                                     verbose=True)

In [43]:
'Action: Search[Colorado orogeny]'.split("Action: ")[1].strip().replace(']','').split('[')

['Search', 'Colorado orogeny']

In [5]:
template_with_history = """Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{history}

Question: {input}
{agent_scratchpad}"""



template_with_history = """Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}][the input to the action]
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: Final Answer: the final answer to the original input question

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: Final Answer: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.



Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Final Answer: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.


Begin!

Previous conversation history:
{history}

Question: {input}
{agent_scratchpad}"""

In [11]:
template_with_history = """Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}][the input to the action]
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: Final Answer: the final answer to the original input question


Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Final Answer: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.


Begin!

Previous conversation history:
{history}

Question: {input}
{agent_scratchpad}"""

In [12]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [13]:
#llm_chain = LLMChain(llm=qa_chain.llm, prompt=prompt_with_history)
llm_chain = LLMChain(llm=pipe_mpt, prompt=prompt_with_history)


tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)
memory = ConversationBufferWindowMemory(k=5)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    max_iterations=10
)
agent_executor.run("What is the most promising direction for investment for the Vantage Towers?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the most promising direction for investment for the Vantage Towers?",
  "history": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "What is the most promising direction for investment for the Vantage Towers?",
  "history": ""
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nLookup: useful for when you need to answer questions about specific precise information in documents\n\nUse the following format:\n\nQuestion: the input question you must answer\

IndexError: list index out of range

In [7]:
#llm_mpt = Model('mpt-7b-storywriter', temperature=0, max_length=83968).llm

In [7]:
llm_chain = LLMChain(llm=qa_chain.llm, prompt=prompt_with_history)

In [8]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [9]:
memory=ConversationBufferWindowMemory(k=5)

In [10]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    max_iterations=10
)

In [11]:
agent_executor.run("What is the most promising direction for investment for the Vantage Towers?")

/opt/conda/envs/gpt/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the most promising direction for investment for the Vantage Towers?",
  "history": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "What is the most promising direction for investment for the Vantage Towers?",
  "history": ""
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nLookup: useful for when you need to answer questions about specific precise information in documents\n\nUse the following format:\n\nQuestion: the input question you must answer\

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] [56.35s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: This question requires me to know the current state of the market and the company's financial situation. I will need to use my knowledge of the industry and research on the company's performance.\nAction: Lookup[Vantage Towers]",
        "generation_info": null
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [56.35s] Exiting Chain run with output:
{
  "text": "Thought: This question requires me to know the current state of the market and the company's financial situation. I will need to use my knowledge of the industry and research on the company's performance.\nAction: Lookup[Vantage Towers]"
}
[tool/start] [1:chain:AgentExecutor > 4:tool:Lookup] Entering Tool run with input:
"Vantage Towers"
[tool/end] [1:chain:AgentExecutor > 4:tool:Lookup] [3

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [ ]:
# tools

In [ ]:
# AgentType.R

In [ ]:
# langchain.debug = True

In [ ]:
# agent_chain = initialize_agent(
#     tools,
#     llm,
#     agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
#     verbose=True,
#     memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# )

In [ ]:
# agent_chain.run("How can I treat a spained ankle?")

In [ ]:
# template = """Answer the following questions as best you can, but speaking as compasionate specialist in cell towers and all related fields. You have access to the following tools:
#
# {tools}
#
# Use the following format:
#
# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question
#
# Begin! Remember to answer as a compasionate specialist in cell towers and all related fields when giving your final answer.
#
# Previous conversation history:
# {history}
#
# New question: {input}
# {agent_scratchpad}"""